In [1]:
pip install pandas

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3 -> 23.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import pandas as pd
melb_data = pd.read_csv('data/melb_data.csv', sep=',')
melb_df = melb_data.copy()
melb_df.head()





,index,Suburb,Address,Rooms,Type,Price,Method,SellerG,Date,Distance,...,Car,Landsize,BuildingArea,YearBuilt,CouncilArea,Lattitude,Longtitude,Regionname,Propertycount,Coordinates
0,0,Abbotsford,85 Turner St,2,h,1480000.0,S,Biggin,3/12/2016,2.5,...,1.0,202.0,126.0,1970.0,Yarra,-37.7996,144.9984,Northern Metropolitan,4019.0,"-37.7996, 144.9984"
1,1,Abbotsford,25 Bloomburg St,2,h,1035000.0,S,Biggin,4/02/2016,2.5,...,0.0,156.0,79.0,1900.0,Yarra,-37.8079,144.9934,Northern Metropolitan,4019.0,"-37.8079, 144.9934"
2,2,Abbotsford,5 Charles St,3,h,1465000.0,SP,Biggin,4/03/2017,2.5,...,0.0,134.0,150.0,1900.0,Yarra,-37.8093,144.9944,Northern Metropolitan,4019.0,"-37.8093, 144.9944"
3,3,Abbotsford,40 Federation La,3,h,850000.0,PI,Biggin,4/03/2017,2.5,...,1.0,94.0,126.0,1970.0,Yarra,-37.7969,144.9969,Northern Metropolitan,4019.0,"-37.7969, 144.9969"
4,4,Abbotsford,55a Park St,4,h,1600000.0,VB,Nelson,4/06/2016,2.5,...,2.0,120.0,142.0,2014.0,Yarra,-37.8072,144.9941,Northern Metropolitan,4019.0,"-37.8072, 144.9941"


In [3]:
countries_df = pd.DataFrame({
    'country': ['Англия', 'Канада', 'США', 'Россия', 'Украина', 'Беларусь', 'Казахстан'],
    'population': [56.29, 38.05, 322.28, 146.24, 45.5, 9.5, 17.04],
    'square': [133396, 9984670, 9826630, 17125191, 603628, 207600, 2724902]
})
countries_df['plot'] = countries_df['population']/countries_df['square']*1000000



print(countries_df['plot'].mean())

84.93080566562


In [4]:
melb_df = melb_data.copy()
melb_df = melb_df.drop(['index', 'Coordinates'], axis=1)
total_rooms = melb_df['Rooms'] + melb_df['Bedroom'] + melb_df['Bathroom']

melb_df['MeanRoomsSquare'] = melb_df['BuildingArea'] / total_rooms
diff_area = melb_df['BuildingArea'] - melb_df['Landsize']
sum_area = melb_df['BuildingArea'] + melb_df['Landsize']
melb_df['AreaRatio'] = diff_area/sum_area

price_square = melb_df['Price'] **2
melb_df['Date'] = pd.to_datetime(melb_df['Date'], dayfirst=True)
melb_df['MonthSale'] = melb_df['Date'].dt.month
melb_df['MonthSale'].value_counts(normalize=True)
delta_days = melb_df['Date'] - pd.to_datetime('2016-01-01') 
melb_df['AgeBuilding'] = melb_df['Date'].dt.year - melb_df['YearBuilt']
melb_df = melb_df.drop('YearBuilt', axis=1)
melb_df['WeekdaySale'] = melb_df['Date'].dt.dayofweek
weekend_count = melb_df[melb_df['WeekdaySale'] > 4].count()


#создание и преобразование столбцов с помощью функций
#создали список сторон света, проверили, есть ли в каждом адресе сторона света, если да, то выбрали второй с конца элемент (пропустили сторону света)
#если нет, то выбрали 1й с конца элемент, получили обьект сириес с подтипом улицы
def get_street_type(adress):
    exclude_list = ['N', 'S', 'W', 'E']
    addres_list = adress.split(' ')
    street_type = addres_list[-1]
    if street_type in exclude_list:
        street_type = addres_list[-2]
    return street_type

street_types = melb_df['Address'].apply(get_street_type)

#извлечём 10 самых популярных значений, остальные назовёт other

popular_stypes = street_types.value_counts().nlargest(10).index #применили метод nlargest, кот возвращает n наибольших значений, извлекли их названия с помощью index
#введём лямбда функцию, кот будет проверять, еслить ли строка в этом перечне, если да, то оставлять её неизменной, если нет, то менять на other
melb_df['StreetType'] = street_types.apply(lambda x: x if x in popular_stypes else 'other')
#удалим столбец адресс, он теперь не нужен
melb_df = melb_df.drop("Address", axis=1)

#задание 4.2
def get_weekend(weekday):
   
    if weekday == 5 or weekday == 6: 
        weekday = 1
    else:
        weekday = 0
    return weekday

melb_df['Weekend'] = melb_df['WeekdaySale'].apply(get_weekend)                       
day_1_0 = melb_df.loc[melb_df['Weekend']==1] #создала новую таблицу, где в столбще день будут только выходные
day_1_0['Price'].mean() #вывела среднее по столбцу цена в новой таблице

#задание 4.3
popular_company = melb_df['SellerG'].value_counts().nlargest(49).index
melb_df['SellerG'] = melb_df['SellerG'].apply(lambda x: x if x in popular_company else 'other')
nelson = melb_df.loc[melb_df['SellerG']=='Nelson']
nelson_min = nelson['Price'].min()
other = melb_df.loc[melb_df['SellerG']=='other']
other_min = other['Price'].min()
otvet = nelson_min/other_min
#сначала определила 49 самых частовстречающихся компаний, через лямбда функцию оставила их, остальные назвала other, 
#отфильтровала Нельсон, создала новую таблицу и по цене вывела мин. Тоже самое сделала с other. Затем вывела на сколько мин цены отличаются





print(otvet)



1.297709923664122


In [5]:
#определяем число уникальных категорий в каждом столбце таблицы
#создаю вспомогательную таблицу в которой строки будут названия столбцов исходной табл, а столбцами количество уникальных категорий
#создаём пустой список
unicue_list = []
for col in melb_df.columns:
    item = (col, melb_df[col].nunique(), melb_df[col].dtypes)
    unicue_list.append(item)
#пробегаемся по именам столбцов в таблице, создаём кортеж для будущих столбцов новой таблицы
#добавляем кортеж в список
unique_counts = pd.DataFrame(unicue_list, columns = ['Columns_Name', 'Num_Unique', 'Type']).sort_values(by = 'Num_Unique', ignore_index = True)
#создаём новую таблицу, строками которой значения в первом списке, а столбцами - 'Columns_Name', 'Num_Unique', 'Type'. Сортируем её тут же по столбцу'Num_Unique'

#делаем преобразование столбцов к типу данных category (где колво уник знач менее 150, и исключая количественные)
cols_to_exclude = ['Date', 'Rooms', 'Bedroom', 'Car'] #список столбцов, которые мы не берем во внимание
max_unique_count = 150 #задаём максимальное число уникальных категорий
for col in melb_df.columns: #цикл по именам столбцов
    if melb_df[col].nunique() <max_unique_count and col not in cols_to_exclude: #проверяем условие (число ун кат меньее 150 и исключаем количественные)
        melb_df[col] = melb_df[col].astype('category') #преобразуем тип выбранных столбцов на category
display(melb_df.info())




<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13580 entries, 0 to 13579
Data columns (total 26 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   Suburb           13580 non-null  object        
 1   Rooms            13580 non-null  int64         
 2   Type             13580 non-null  category      
 3   Price            13580 non-null  float64       
 4   Method           13580 non-null  category      
 5   SellerG          13580 non-null  category      
 6   Date             13580 non-null  datetime64[ns]
 7   Distance         13580 non-null  float64       
 8   Postcode         13580 non-null  int64         
 9   Bedroom          13580 non-null  float64       
 10  Bathroom         13580 non-null  category      
 11  Car              13580 non-null  float64       
 12  Landsize         13580 non-null  float64       
 13  BuildingArea     13580 non-null  float64       
 14  CouncilArea      12211 non-null  categ

None

In [55]:
#задание 5.2
#melb_df.info()
#popular_suburb = melb_df['Suburb'].value_counts().nlargest(119).index
#melb_df['Suburb'] = melb_df['Suburb'].apply(lambda x: x if x in popular_suburb else 'other')
#melb_df['Suburb'] = melb_df['Suburb'].astype('category')
melb_df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13580 entries, 0 to 13579
Data columns (total 26 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   Suburb           13580 non-null  category      
 1   Rooms            13580 non-null  int64         
 2   Type             13580 non-null  category      
 3   Price            13580 non-null  float64       
 4   Method           13580 non-null  category      
 5   SellerG          13580 non-null  category      
 6   Date             13580 non-null  datetime64[ns]
 7   Distance         13580 non-null  float64       
 8   Postcode         13580 non-null  int64         
 9   Bedroom          13580 non-null  float64       
 10  Bathroom         13580 non-null  category      
 11  Car              13580 non-null  float64       
 12  Landsize         13580 non-null  float64       
 13  BuildingArea     13580 non-null  float64       
 14  CouncilArea      12211 non-null  categ

In [55]:
#велоспорт
import pandas as pd
citibike = pd.read_csv('data/citibike-tripdata.csv', sep=',')
citibike_drop = citibike.copy()
citibike_drop = citibike_drop.drop(['start station id', 'end station id'], axis=1)
ggg = 2018 - citibike_drop['birth year']
citibike_drop['age'] = ggg
citibike_drop["stoptime"] = pd.to_datetime(citibike_drop['stoptime'])
citibike_drop["starttime"] = pd.to_datetime(citibike_drop['starttime'])
citibike_drop['trip duration'] = citibike_drop['stoptime'] - citibike_drop['starttime']
citibike_drop['weekend'] = citibike_drop['starttime'].dt.day_of_week
def week(day):
    if day < 5: day = 0
    else: day = 1
    return day
citibike_drop['weekend'] = citibike_drop['weekend'].apply(week)

citibike_drop["time_of_day"] = citibike_drop['starttime'].dt.hour.astype('int64')


def timeofday(time):
    if time >= 13 and time <= 18: 
        time = 'night'
    #if i >= 7 and i <= 12: 
        # = 'morning'
    #if 12 < time <= 18: 
        #time = 'day'
    #if 18 < time <= 23: 
        #time = 'evening'
    return time

citibike_drop["time_of_day"] = citibike_drop["time_of_day"].apply(timeofday)
    
    
    
    
display(citibike_drop["time_of_day"].value_counts())


night    143012
12        19157
8         17807
19        17136
11        16821
9         15776
10        15007
20        11168
7         10962
21         7827
22         6035
6          5325
23         4207
0          3360
1          1977
5          1592
2          1299
3           793
4           739
Name: time_of_day, dtype: int64

In [114]:
import pandas as pd
melb_df = pd.read_csv('data/melb_data_fe.csv')

melb_df['Date'] = pd.to_datetime(melb_df['Date'], dayfirst=True)
#melb_df['Date'] = melb_df['Date'].dt.quarter

#делаем преобразование столбцов к типу данных category (где колво уник знач менее 150, и исключая количественные)
cols_to_exclude = ['Date', 'Rooms', 'Bedroom', 'Bathroom', 'Car'] #список столбцов, которые мы не берем во внимание
max_unique_count = 150 #задаём максимальное число уникальных категорий
for col in melb_df.columns: #цикл по именам столбцов
    if melb_df[col].nunique() <max_unique_count and col not in cols_to_exclude: #проверяем условие (число ун кат меньее 150 и исключаем количественные)
        melb_df[col] = melb_df[col].astype('category') #преобразуем тип выбранных столбцов на category
melb_df.sort_values(by='Price').head(10)
melb_df.sort_values(by='Date', ascending=False)
melb_df.sort_values(by=['Distance', 'Price']).loc[::10, ['Distance', 'Price']]

#Найдём информацию о таунхаусах (Type), проданных компанией (SellerG) McGrath, у которых коэффициент соотношения площадей здания и участка (AreaRatio) меньше -0.8. Результат отсортируем по дате продажи (Date) в порядке возрастания, а после проведём сортировку по убыванию коэффициента соотношения площадей. Также обновим старые индексы на новые, установив параметр ignore_index на True. Для наглядности результата выберем из таблицы только столбцы Data и AreaRatio:
mask1 = melb_df['AreaRatio'] < -0.8 #в столбце area ratio выставляем маску-фильтр на больше 0.8 
mask2 = melb_df['Type'] == 'townhouse' #в столбце type выставляем фильтр таунхаус
mask3 = melb_df['SellerG'] == 'McGrath'#в столбце селлер - фирму McGrath
melb_df[mask1 & mask2 & mask3].sort_values(  #применяем маски к таблице
    by=['Date', 'AreaRatio'],
    ascending=[True, False],
    ignore_index=True
).loc[:, ['Date', 'AreaRatio']] #выводим на экран только эти 2 столбца


melb_df.sort_values(by='AreaRatio', ascending=False, ignore_index = True)

display(melb_df)

,Suburb,Rooms,Type,Price,Method,SellerG,Date,Distance,Postcode,Bedroom,...,Longtitude,Regionname,Propertycount,MeanRoomsSquare,AreaRatio,MonthSale,AgeBuilding,WeekdaySale,StreetType,Weekend
0,Abbotsford,2,house,1480000.0,S,Biggin,2016-03-12,2.5,3067,2,...,144.99840,Northern Metropolitan,4019,25.200000,-0.231707,3,46,5,St,1
1,Abbotsford,2,house,1035000.0,S,Biggin,2016-04-02,2.5,3067,2,...,144.99340,Northern Metropolitan,4019,15.800000,-0.327660,4,116,5,St,1
2,Abbotsford,3,house,1465000.0,SP,Biggin,2017-04-03,2.5,3067,3,...,144.99440,Northern Metropolitan,4019,18.750000,0.056338,4,117,0,St,0
3,Abbotsford,3,house,850000.0,PI,Biggin,2017-04-03,2.5,3067,3,...,144.99690,Northern Metropolitan,4019,15.750000,0.145455,4,47,0,other,0
4,Abbotsford,4,house,1600000.0,VB,Nelson,2016-04-06,2.5,3067,3,...,144.99410,Northern Metropolitan,4019,17.750000,0.083969,4,2,2,St,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13575,other,4,house,1245000.0,S,Barry,2017-08-26,16.7,3150,4,...,145.16761,South-Eastern Metropolitan,7392,12.600000,-0.676093,8,36,5,Cr,1
13576,Williamstown,3,house,1031000.0,SP,Williams,2017-08-26,6.8,3016,3,...,144.87904,Western Metropolitan,6380,16.625000,-0.429185,8,22,5,Dr,1
13577,Williamstown,3,house,1170000.0,S,Raine,2017-08-26,6.8,3016,3,...,144.88738,Western Metropolitan,6380,15.750000,-0.551601,8,20,5,St,1
13578,Williamstown,4,house,2500000.0,PI,Sweeney,2017-08-26,6.8,3016,4,...,144.89299,Western Metropolitan,6380,17.444444,-0.693060,8,97,5,St,1


In [118]:
melb_df = pd.read_csv('data/melb_data_fe.csv')
melb_df3 = melb_df.copy()

mask2 = melb_df3['Type'] == 'townhouse' #в столбце type выставляем фильтр таунхаус
mask3 = melb_df['Rooms'] > 2#в столбце селлер - фирму McGrath
melb_df3[mask2 & mask3].sort_values(  #применяем маски к таблице
    by=['Rooms', 'MeanRoomsSquare'],
    ascending=[True, False],
    ignore_index=True
).loc[18] #выводим на экран только эти 2 столбца








Suburb                     Brighton East
Rooms                                  3
Type                           townhouse
Price                          1300000.0
Method                                PI
SellerG                           Buxton
Date                          2016-09-17
Distance                            10.7
Postcode                            3187
Bedroom                                3
Bathroom                               2
Car                                    2
Landsize                           337.0
BuildingArea                       222.0
CouncilArea                      Bayside
Lattitude                       -37.9196
Longtitude                      145.0205
Regionname         Southern Metropolitan
Propertycount                       6938
MeanRoomsSquare                    27.75
AreaRatio                      -0.205725
MonthSale                              9
AgeBuilding                           16
WeekdaySale                            5
StreetType      

In [125]:
melb_df.groupby('MonthSale')['Price'].agg( #так как мы считаем несколько показателей для одного столбца, в результате мы получаем объект DataFrame.
    ['count', 'mean', 'max']
).sort_values(by='count', ascending=False)

,count,mean,max
MonthSale,,,
8,1850,1.056371e+06,6500000.0
7,1835,9.314698e+05,9000000.0
5,1644,1.097807e+06,8000000.0
6,1469,1.068981e+06,7650000.0
3,1408,1.146762e+06,5600000.0
4,1246,1.050479e+06,5500000.0
9,1188,1.126349e+06,6400000.0
10,854,1.135970e+06,6250000.0
11,750,1.142503e+06,5050000.0


In [127]:
melb_df.groupby('MonthSale')['Price'].agg('describe') #Если вам нужна полная информация обо всех основных статистических характеристиках внутри каждой группы, вы можете воспользоваться методом agg(), передав в качестве его параметра строку 'describe':

,count,mean,std,min,25%,50%,75%,max
MonthSale,,,,,,,,
1,278.0,9.397921e+05,577668.924214,170000.0,570500.0,795000.0,1111250.0,5200000.0
2,333.0,1.169051e+06,671564.357417,131000.0,710000.0,1020000.0,1478000.0,4735000.0
3,1408.0,1.146762e+06,709573.596867,85000.0,680000.0,945000.0,1400000.0,5600000.0
4,1246.0,1.050479e+06,591892.902979,145000.0,655000.0,905500.0,1298750.0,5500000.0
5,1644.0,1.097807e+06,668492.867996,145000.0,650000.0,905000.0,1371250.0,8000000.0
6,1469.0,1.068981e+06,606010.069052,222000.0,660000.0,900000.0,1325000.0,7650000.0
7,1835.0,9.314698e+05,537390.803161,190000.0,586750.0,800000.0,1150000.0,9000000.0
8,1850.0,1.056371e+06,619617.476541,160000.0,635000.0,892000.0,1310000.0,6500000.0
9,1188.0,1.126349e+06,608734.690742,170000.0,725000.0,980000.0,1360000.0,6400000.0


In [126]:
melb_df.groupby('Type')['Price'].mean() #Применим агрегирующую функцию среднего к результату работы groupby(). В качестве столбца для группировки возьмём столбец типа объекта недвижимости (Type):

Type
house        1.242665e+06
townhouse    9.337351e+05
unit         6.051275e+05
Name: Price, dtype: float64

In [142]:
melb_df['Date'] = pd.to_datetime(melb_df['Date'], dayfirst=True)
mask1 = melb_df['Date'] > '2017-05-01'
mask2 = melb_df['Date'] < '2017-9-1'
melb_df[mask1 & mask2]
melb_df.groupby('SellerG')['Price'].sum().sort_values()


SellerG
LITTLE           2.199300e+07
Burnham          2.614650e+07
Moonee           2.878050e+07
Thomson          2.981600e+07
Bells            3.038450e+07
Alexkarbon       3.465559e+07
Buckingham       3.881160e+07
Eview            4.065124e+07
McDonald         4.205350e+07
Purplebricks     4.360900e+07
Philip           4.703580e+07
HAR              4.818300e+07
O'Brien          4.999551e+07
C21              5.350800e+07
Douglas          6.429010e+07
Rendina          6.600628e+07
Cayzer           6.867000e+07
RW               7.013100e+07
Nick             7.531900e+07
Collins          7.674250e+07
Chisholm         7.834425e+07
Love             7.894540e+07
Raine            8.946020e+07
YPA              9.273035e+07
Stockdale        1.064608e+08
Williams         1.156803e+08
Village          1.235205e+08
Harcourts        1.328355e+08
Gary             1.671684e+08
Sweeney          1.805233e+08
Hodges           2.062959e+08
Miles            2.130829e+08
Jas              2.143306e+08
Ka

In [143]:
melb_df.groupby(['Rooms', 'Type'])['Price'].mean().unstack()

Type,house,townhouse,unit
Rooms,,,
1,8.668655e+05,5.927045e+05,3.899289e+05
2,1.017238e+06,7.101585e+05,6.104905e+05
3,1.109233e+06,9.847087e+05,8.505963e+05
4,1.462283e+06,1.217092e+06,1.037476e+06
5,1.877327e+06,1.035000e+06,NaN
6,1.869508e+06,NaN,5.200000e+05
7,1.920700e+06,NaN,NaN
8,1.510286e+06,NaN,2.250000e+06
10,9.000000e+05,NaN,NaN


In [128]:
melb_df.groupby('Regionname')['SellerG'].agg(
    		['nunique', set]
) 
#ычислить число уникальных риелторских компаний в зависимости от региона, чтобы понять, в каких регионах конкуренция на рынке недвижимости меньше. Это можно сделать, передав в параметр метода agg() строку 'nunique'

,nunique,set
Regionname,,
Eastern Metropolitan,26,"{Kay, McGrath, Stockdale, Love, Woodards, Gary..."
Eastern Victoria,11,"{C21, hockingstuart, other, McGrath, Eview, Ha..."
Northern Metropolitan,40,"{Kay, LITTLE, Village, McGrath, McDonald, Brad..."
Northern Victoria,11,"{LITTLE, hockingstuart, other, McDonald, McGra..."
South-Eastern Metropolitan,25,"{McGrath, Stockdale, Woodards, Gary, Hodges, B..."
Southern Metropolitan,38,"{Kay, LITTLE, Cayzer, McGrath, Stockdale, Jas,..."
Western Metropolitan,34,"{Village, McGrath, McDonald, Brad, Stockdale, ..."
Western Victoria,6,"{hockingstuart, other, Raine, Ray, YPA, HAR}"


In [145]:
melb_df.pivot_table(
    values='Price',
    index='Regionname',
    columns='Weekend',
    aggfunc='count'
) #проанализируем продажи в каждом из регионов в зависимости от того, будний был день или выходной

Weekend,0,1
Regionname,,
Eastern Metropolitan,447,1024
Eastern Victoria,13,40
Northern Metropolitan,1258,2632
Northern Victoria,11,30
South-Eastern Metropolitan,123,327
Southern Metropolitan,1534,3161
Western Metropolitan,960,1988
Western Victoria,8,24


In [146]:
melb_df.pivot_table(
    values='Landsize',
    index='Regionname',
    columns='Type',
    aggfunc=['median', 'mean'],
    fill_value=0
)
#ак зависит средняя и медианная площадь участка (Landsize) от типа объекта (Type) и его региона (Regionname).

median                        mean              \
Type                        house townhouse unit        house   townhouse   
Regionname                                                                  
Eastern Metropolitan        674.0     233.5  203   717.422847  269.440678   
Eastern Victoria            843.0       0.0  230  3108.960000    0.000000   
Northern Metropolitan       459.5     134.0    0   619.249092  317.325733   
Northern Victoria           724.0       0.0    0  3355.463415    0.000000   
South-Eastern Metropolitan  630.5     240.0  199   664.306701  212.160000   
Southern Metropolitan       586.0     246.0    0   569.643881  278.858824   
Western Metropolitan        531.0     198.0   62   507.883406  244.560669   
Western Victoria            599.5       0.0    0   655.500000    0.000000   

                                        
Type                              unit  
Regionname                              
Eastern Metropolitan        330.444444  
Eastern Victoria            295.333333  
Northern Metropolitan       495.026538  
Northern Victoria             0.000000  
South-Eastern Metropolitan  357.864865  
Southern Metropolitan       466.380245  
Western Metropolitan        557.637232  
Western Victoria              0.000000

In [147]:
melb_df.pivot_table(
    values='Price',
    index=['Method','Type'],
    columns='Regionname',
    aggfunc='median',
    fill_value=0
)
#Давайте построим таблицу, в которой по индексам будут располагаться признаки метода продажи (Method) и типа объекта (Type), по столбцам — наименование региона (Regionname), а на пересечении строк и столбцов будет стоять медианная цена объекта (Price):

Regionname        Eastern Metropolitan  Eastern Victoria  \
Method Type                                                
PI     house                   1244000            780000   
       townhouse                760000                 0   
       unit                     650000                 0   
S      house                   1127000            675000   
       townhouse                828000                 0   
       unit                     645750            492000   
SA     house                    932500            950000   
       townhouse                807500                 0   
       unit                          0                 0   
SP     house                   1050000            672500   
       townhouse                910000                 0   
       unit                     515000            400000   
VB     house                   1100000            712500   
       townhouse                892500                 0   
       unit                     500000                 0   

Regionname        Northern Metropolitan  Northern Victoria  \
Method Type                                                  
PI     house                     900000             500000   
       townhouse                 632500                  0   
       unit                      410000                  0   
S      house                     920000             555000   
       townhouse                 750000                  0   
       unit                      525500                  0   
SA     house                     817500             540000   
       townhouse                 425000                  0   
       unit                      616000                  0   
SP     house                     900000             521000   
       townhouse                 690000                  0   
       unit                      470000                  0   
VB     house                    1050000             690000   
       townhouse                 640000                  0   
       unit                      450000                  0   

Regionname        South-Eastern Metropolitan  Southern Metropolitan  \
Method Type                                                           
PI     house                          865000                1725000   
       townhouse                     1190000                1055000   
       unit                           525000                 571250   
S      house                          883300                1611000   
       townhouse                      875000                1135000   
       unit                           606000                 655000   
SA     house                          880000                1390000   
       townhouse                           0                1141000   
       unit                                0                 580000   
SP     house                          770000                1521750   
       townhouse                      800000                1162500   
       unit                           601000                 550000   
VB     house                          850000                1800000   
       townhouse                           0                1250000   
       unit                           700000                 500000   

Regionname        Western Metropolitan  Western Victoria  
Method Type                                               
PI     house                    870000            630000  
       townhouse                670000                 0  
       unit                     360000                 0  
S      house                    870000            397500  
       townhouse                729000                 0  
       unit                     489000                 0  
SA     house                    772500                 0  
       townhouse                467500                 0  
       unit                     571000                 0  
SP     house                    865000            360000  
       t

In [148]:
pivot = melb_df.pivot_table(
    values='Landsize',
    index='Regionname',
    columns='Type',
    aggfunc=['median', 'mean'],
    fill_value=0
)
#Запишем сводную таблицу, которую мы создавали ранее в переменную pivot
display(pivot['mean']['unit'])
#из таблицы pivot мы можем получить средние значения площадей участков для типа здания unit, просто последовательно обратившись по имени столбцов

Regionname
Eastern Metropolitan          330.444444
Eastern Victoria              295.333333
Northern Metropolitan         495.026538
Northern Victoria               0.000000
South-Eastern Metropolitan    357.864865
Southern Metropolitan         466.380245
Western Metropolitan          557.637232
Western Victoria                0.000000
Name: unit, dtype: float64

In [150]:
melb_df.pivot_table(
    values='Price',
    index='SellerG',
    columns='Type',
    aggfunc='median'
)

Type,house,townhouse,unit
SellerG,,,
Alexkarbon,1036250.0,935000.0,560000.0
Barry,830000.0,727500.0,502000.0
Bells,692500.0,430500.0,457500.0
Biggin,1151750.0,675000.0,548000.0
Brad,875000.0,700000.0,465000.0
Buckingham,830000.0,820000.0,553000.0
Burnham,922500.0,NaN,305000.0
Buxton,1323750.0,1030000.0,670000.0
C21,836500.0,768750.0,625500.0
